# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,18.91,93,20,3.60,US,1712488343
1,1,ventspils,57.3894,21.5606,14.96,77,0,7.72,LV,1712488343
2,2,grytviken,-54.2811,-36.5092,4.57,98,100,5.84,GS,1712488343
3,3,albany,42.6001,-73.9662,3.06,80,76,0.89,US,1712488343
4,4,sechura,-5.5569,-80.8222,22.26,78,100,3.63,PE,1712488343


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df['Max Temp'] < 23) &
                                (city_data_df['Max Temp'] > 18) &
                                (city_data_df['Cloudiness'] <= 30)& 
                                (city_data_df['Humidity'] < 80) &
                                (city_data_df['Country'] != 'RU') &
                                (city_data_df['Country'] != 'CN') &
                                (city_data_df['Country'] != 'AU')
                                ]

# Drop any rows with null values
ideal_cities.dropna(inplace=True)

# Display sample data
ideal_cities

C:\Users\cadkm\AppData\Local\Temp\ipykernel_37924\3052718602.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,lata,40.1629,-8.3327,19.49,56,14,2.00,PT,1712488345
87,87,nouadhibou,20.9310,-17.0347,22.99,68,0,6.69,MR,1712488356
132,132,port said,31.2565,32.2841,19.19,72,11,4.69,EG,1712488363
202,202,tazacorte,28.6290,-17.9293,20.54,77,20,5.14,ES,1712488376
275,275,upington,-28.4478,21.2561,21.63,24,0,5.88,ZA,1712488249
281,281,coahuayana de hidalgo,18.7000,-103.6583,20.26,53,16,1.52,MX,1712488387
346,346,vodice,43.7608,15.7828,21.98,67,4,6.11,HR,1712488395
364,364,pokrovsk,48.2820,37.1758,19.60,36,23,3.03,UA,1712488398
399,399,saint-francois,46.4154,3.9054,20.38,60,0,1.54,FR,1712488403
406,406,tukrah,32.5341,20.5791,20.59,54,0,5.78,LY,1712488404


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities.filter(items= ['City', 'Country', 'Lat', 'Lng', 'Humidity'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,lata,PT,40.1629,-8.3327,56,
87,nouadhibou,MR,20.9310,-17.0347,68,
132,port said,EG,31.2565,32.2841,72,
202,tazacorte,ES,28.6290,-17.9293,77,
275,upington,ZA,-28.4478,21.2561,24,
281,coahuayana de hidalgo,MX,18.7000,-103.6583,53,
346,vodice,HR,43.7608,15.7828,67,
364,pokrovsk,UA,48.2820,37.1758,36,
399,saint-francois,FR,46.4154,3.9054,60,
406,tukrah,LY,32.5341,20.5791,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lata - nearest hotel: Residencial Botânico
nouadhibou - nearest hotel: Hotel valencia
port said - nearest hotel: The New Continental Hotel
tazacorte - nearest hotel: App Leyma
upington - nearest hotel: Die Eiland Holiday Ressort
coahuayana de hidalgo - nearest hotel: No hotel found
vodice - nearest hotel: Scala
pokrovsk - nearest hotel: Дружба
saint-francois - nearest hotel: Chez Lily
tukrah - nearest hotel: No hotel found
mazamet - nearest hotel: No hotel found
zhovti vody - nearest hotel: No hotel found
essaouira - nearest hotel: Essaouira Wind Palace
azemmour - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
colonia - nearest hotel: Wasserturm Hotel Cologne
mezhova - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
16,lata,PT,40.1629,-8.3327,56,Residencial Botânico
87,nouadhibou,MR,20.9310,-17.0347,68,Hotel valencia
132,port said,EG,31.2565,32.2841,72,The New Continental Hotel
202,tazacorte,ES,28.6290,-17.9293,77,App Leyma
275,upington,ZA,-28.4478,21.2561,24,Die Eiland Holiday Ressort
281,coahuayana de hidalgo,MX,18.7000,-103.6583,53,No hotel found
346,vodice,HR,43.7608,15.7828,67,Scala
364,pokrovsk,UA,48.2820,37.1758,36,Дружба
399,saint-francois,FR,46.4154,3.9054,60,Chez Lily
406,tukrah,LY,32.5341,20.5791,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    color = "City",
    hover_cols= ["Country", "Hotel Name"])


# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)